# AgentCore Runtime에서 대용량 멀티모달 Payload 처리하기

## 개요

이 튜토리얼은 Amazon Bedrock AgentCore Runtime이 Excel 파일 및 이미지와 같은 멀티모달 콘텐츠를 포함하여 최대 100MB의 대용량 payload를 처리하는 방법을 보여줍니다. AgentCore Runtime은 풍부한 미디어 콘텐츠와 대규모 데이터셋을 원활하게 처리하도록 설계되었습니다.

### 튜토리얼 세부 정보

|정보| 세부사항|
|:--------------------|:---------------------------------------------------------------------------------|
| 튜토리얼 유형       | 대용량 Payload 및 멀티모달 처리|
| Agent 유형          | Single         |
| Agentic Framework   | Strands Agents |
| LLM model           | Anthropic Claude Haiku 4.5 |
| 튜토리얼 구성요소 | 대용량 파일 처리, 이미지 분석, Excel 데이터 처리 |
| 튜토리얼 분야   | 데이터 분석 및 멀티모달 AI                                                   |
| 예제 복잡도  | 중급                                                                     |
| 사용 SDK            | Amazon BedrockAgentCore Python SDK|

### 주요 기능

* **대용량 Payload 지원**: 최대 100MB 크기의 파일 처리
* **멀티모달 처리**: Excel 파일, 이미지 및 텍스트를 동시에 처리
* **데이터 분석**: 구조화된 데이터 및 시각적 콘텐츠에서 인사이트 추출
* **Base64 인코딩**: JSON payload를 통한 바이너리 데이터의 안전한 전송

## 사전 요구사항

* Python 3.10+
* AWS 자격 증명 구성
* Docker 실행 중
* 테스트용 샘플 Excel 파일 및 이미지

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## 샘플 데이터 파일 생성

대용량 payload 처리를 시연하기 위해 샘플 Excel 및 이미지 파일을 생성해 보겠습니다:

In [ ]:
import pandas as pd
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import os

# 샘플 판매 데이터가 포함된 대용량 Excel 파일 생성
np.random.seed(42)
data = {
    'Date': pd.date_range('2023-01-01', periods=1000, freq='h'),  # 시간 단위로 1000개 날짜 생성
    'Product': np.random.choice(['Widget A', 'Widget B', 'Widget C', 'Gadget X', 'Gadget Y'], 1000),
    'Sales': np.random.randint(1, 1000, 1000),
    'Revenue': np.random.uniform(10.0, 5000.0, 1000),
    'Region': np.random.choice(['North', 'South', 'East', 'West'], 1000),
    'Customer_ID': np.random.randint(1000, 9999, 1000)
}

df = pd.DataFrame(data)
df.to_excel('large_sales_data.xlsx', index=False)

# 샘플 차트 이미지 생성
img = Image.new('RGB', (600, 500), color='white')
draw = ImageDraw.Draw(img)

# 간단한 막대 차트 그리기
products = ['Widget A', 'Widget B', 'Widget C', 'Gadget X', 'Gadget Y']
values = [250, 180, 320, 150, 280]
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7']

max_value = max(values)
bar_width = 120
start_x = 100

for i, (product, value, color) in enumerate(zip(products, values, colors)):
    x = start_x + i * (bar_width + 20)
    height = int((value / max_value) * 400)  # 최대값 대비 비율로 높이 계산
    y = 500 - height  # 하단에서부터 그리기 위해 y 좌표 계산
    
    # 막대 그리기
    draw.rectangle([x, y, x + bar_width, 500], fill=color)
    
    # 레이블 추가 (폰트 없이 간소화)
    draw.text((x + 10, 510), product[:8], fill='black')
    draw.text((x + 10, y - 20), str(value), fill='black')

draw.text((300, 50), 'Sales Performance by Product', fill='black')
img.save('sales_chart.png')

# 파일 크기 확인
excel_size = os.path.getsize('large_sales_data.xlsx') / (1024 * 1024)  # MB 단위로 변환
image_size = os.path.getsize('sales_chart.png') / (1024 * 1024)  # MB 단위로 변환

print(f"Excel file size: {excel_size:.2f} MB")
print(f"Image file size: {image_size:.2f} MB")
print(f"Total payload size: {excel_size + image_size:.2f} MB")

## 멀티모달 Agent 생성

대용량 payload에서 Excel 파일과 이미지를 모두 처리할 수 있는 agent를 생성해 보겠습니다:

In [ ]:
%%writefile multimodal_data_agent.py
from strands import Agent, tool
from strands.models import BedrockModel
import pandas as pd
import base64
import io
import json
from bedrock_agentcore.runtime import BedrockAgentCoreApp

app = BedrockAgentCoreApp()

# 모델 및 agent 초기화
model_id = "global.anthropic.claude-haiku-4-5-20251001-v1:0"
model = BedrockModel(
    model_id=model_id,
    max_tokens=16000
)

agent = Agent(
    model=model,
    system_prompt="""
    You are a data analysis assistant that can process large Excel files and images.
    When given multi-modal data, analyze both the structured data and visual content,
    then provide comprehensive insights combining both data sources.
    """
)

@app.entrypoint
def multimodal_data_processor(payload, context):
    """
    Process large multi-modal payloads containing Excel data and images.
    
    Args:
        payload: Contains prompt, excel_data (base64), image_data (base64)
        context: Runtime context information
    
    Returns:
        str: Analysis results from both data sources
    """
    prompt = payload.get("prompt", "Analyze the provided data.")
    excel_data = payload.get("excel_data", "")
    image_data = payload.get("image_data", "")
    
    print(f"=== Large Payload Processing ===")
    print(f"Session ID: {context.session_id}")
    
    if excel_data:
        print(f"Excel data size: {len(excel_data) / 1024 / 1024:.2f} MB")
    if image_data:
        print(f"Image data size: {len(image_data) / 1024 / 1024:.2f} MB")
    print(f"Excel data {excel_data}")
    print(f"Image data {image_data}")
    print(f"=== Processing Started ===")
    # base64 문자열을 bytes로 디코딩
    excel_bytes = base64.b64decode(excel_data)
    # base64 문자열을 bytes로 디코딩
    image_bytes = base64.b64decode(image_data)
    
    # 데이터 컨텍스트가 포함된 향상된 프롬프트
    enhanced_prompt = f"""{prompt}
    Please analyze both data sources and provide insights.
    """
    
    # agent에 멀티모달 콘텐츠 전달 (document, image, text)
    response = agent(
        [{
            "document": {
                "format": "xlsx",
                "name": "excel_data",
                "source": {
                    "bytes": excel_bytes
                }
            }
        },
        {
            "image": {
                "format": "png",
                "source": {
                    "bytes": image_bytes
                }
            }
        },
        {
            "text": enhanced_prompt
        }]
    )
    return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()

## 인프라 설정 및 Agent 배포

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session

boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()

# AgentCore Runtime 설정 (실행 역할 및 ECR 자동 생성)
response = agentcore_runtime.configure(
    entrypoint="multimodal_data_agent.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name="multimodal_data_agent"
)

launch_result = agentcore_runtime.launch()

In [ ]:
import time

status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']

# 배포가 완료될 때까지 대기 (10초마다 상태 확인)
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(f"Deployment status: {status}")

print(f"Final status: {status}")

## 대용량 멀티모달 Payload 테스트

이제 Excel 데이터와 이미지를 모두 포함하는 대용량 payload로 agent를 테스트해 보겠습니다:

In [ ]:
import base64
import uuid
import json
from IPython.display import Markdown, display

# 파일을 base64로 인코딩
with open('large_sales_data.xlsx', 'rb') as f:
    excel_base64 = base64.b64encode(f.read()).decode('utf-8')

with open('sales_chart.png', 'rb') as f:
    image_base64 = base64.b64encode(f.read()).decode('utf-8')

# 대용량 payload 생성
large_payload = {
    "prompt": "Analyze the sales data from the Excel file and correlate it with the chart image. Provide insights on sales performance and trends.",
    "excel_data": excel_base64,
    "image_data": image_base64
}

session_id = str(uuid.uuid4())
print(f"📊 Processing large multi-modal payload...")
print(f"📋 Session ID: {session_id}")
print(f"📄 Excel size: {len(excel_base64) / 1024 / 1024:.2f} MB")
print(f"🖼️ Image size: {len(image_base64) / 1024 / 1024:.2f} MB")
print(f"📦 Total payload: {len(json.dumps(large_payload)) / 1024 / 1024:.2f} MB\n")

# 대용량 payload로 agent 호출
invoke_response = agentcore_runtime.invoke(
    large_payload,
    session_id=session_id
)
final_response = ""
# 스트리밍 응답 수집
for r in invoke_response['response']:
    final_response += r
response_data = final_response
display(Markdown(response_data))

## 리소스 정리

In [ ]:
import boto3

# AWS 리소스 정리
agentcore_control_client = boto3.client('bedrock-agentcore-control', region_name=region)
ecr_client = boto3.client('ecr', region_name=region)

# AgentCore Runtime 삭제
runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)

# ECR 리포지토리 삭제 (force=True로 이미지 포함 삭제)
ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

# 로컬 파일 정리
os.remove('large_sales_data.xlsx')
os.remove('sales_chart.png')

print("✅ Cleanup completed!")

# 축하합니다!

Amazon Bedrock AgentCore Runtime으로 대용량 멀티모달 payload 처리를 성공적으로 시연했습니다!

## 학습한 내용:

### 대용량 Payload 처리
* **100MB 지원**: AgentCore Runtime은 최대 100MB의 payload를 처리할 수 있습니다
* **Base64 인코딩**: JSON payload를 통한 바이너리 데이터의 안전한 전송
* **효율적인 처리**: 대용량 데이터 처리에 최적화된 Runtime

### 멀티모달 기능
* **Excel 분석**: 스프레드시트의 구조화된 데이터 처리
* **이미지 처리**: 시각적 콘텐츠 및 차트 분석
* **통합 분석**: 여러 데이터 유형의 인사이트 상관관계 분석

### 주요 이점
* **풍부한 데이터 처리**: 복잡한 다중 형식 데이터셋 처리
* **확장 가능한 아키텍처**: 대규모 워크로드를 위해 설계된 Runtime
* **Tool 통합**: 특수 데이터 처리를 위한 커스텀 도구
* **엔터프라이즈 준비**: 민감한 비즈니스 데이터의 안전한 처리

이는 AgentCore Runtime이 여러 데이터 모달리티를 사용하는 엔터프라이즈 규모의 데이터 처리 작업을 처리할 수 있는 능력을 보여주며, 복잡한 비즈니스 인텔리전스 및 데이터 분석 애플리케이션에 이상적입니다.